In [ ]:
import numpy as np
from plot_bezier import plot_bezier
import matplotlib.pyplot as plt

In [ ]:

def cubic_bezier_points(control_points, t_values):
    # Define the characteristic matrix for cubic Bézier curve
    M = np.array([
        [1, 0, 0, 0],
        [-3, 3, 0, 0],
        [3, -6, 3, 0],
        [-1, 3, -3, 1]
    ])
    
    # Compute the points on the Bézier curve for each t value
    bezier_points = []
    for t in t_values:
        T = np.array([1, t, t**2, t**3])
        point = T @ M @ control_points  # Matrix multiplication to get the point
        bezier_points.append(point)
    
    return np.array(bezier_points)

def numeric_distance_integration(control_points, resolution=1000):
    t_values = np.linspace(0, 1, resolution+1)
    bezier_points = cubic_bezier_points(control_points, t_values)
    distance = np.sqrt(np.sum(np.power(bezier_points[:-1,:] - bezier_points[1:,:],2),axis=-1))
    return distance

def cubic_bezier_points_equdistant(control_points, count=20, resolution=1000):
    x = np.linspace(0, 1, resolution)
    y = numeric_distance_integration(control_points, resolution=resolution)
    length = np.sum(y)
    t_values_equidistant = np.interp(np.linspace(0, 1, count),y.cumsum()/length,x,)
    return cubic_bezier_points(control_points, t_values_equidistant)

In [ ]:

# Example control points (P0, P1, P2, P3)
control_points = np.array([
    [1, 1, 2],  # P0
    [8, 8, 2],  # P1
    [8, 5, 8],  # P2
    [8, 2, 8]   # P3
])

# Example t values
t_values = np.linspace(0, 1, 15)  # 10 points from 0 to 1

# Calculate bezier points
bezier_points = cubic_bezier_points(control_points, t_values)

plot_bezier(control_points, bezier_points)

In [ ]:


bezier_points = cubic_bezier_points_equdistant(control_points)
plot_bezier(control_points, bezier_points)

In [ ]:
def cubic_bezier_points_extended(control_points, t_values):
    # Define the characteristic matrix for cubic Bézier curve
    M = np.array([
        [1, 0, 0, 0],
        [-3, 3, 0, 0],
        [3, -6, 3, 0],
        [-1, 3, -3, 1]
    ])
    
    # Calculate the number of segments based on the control points
    n = (len(control_points) - 1) // 3
    
    # Initialize the list to hold the computed points
    bezier_points = []
    
    for t in t_values:
        # Determine which segment this t value falls into
        segment_index = int(t)  # This will floor the t value to the nearest integer
        if segment_index >= n:
            segment_index = n - 1  # Clamp to the last segment for t values out of range
        
        # Normalize t to the local coordinate system of the current segment [0, 1]
        local_t = t - segment_index
        
        # Select the appropriate control points for the current segment
        cp_index = segment_index * 3
        segment_control_points = control_points[cp_index:cp_index+4]
        
        # Compute the T vector for the cubic Bézier curve
        T = np.array([1, local_t, local_t**2, local_t**3])
        
        # Compute the point on the curve for the current t value
        point = T @ M @ segment_control_points  # Matrix multiplication to get the point
        bezier_points.append(point)
    
    return np.array(bezier_points)

def numeric_distance_integration(control_points, resolution=1000):
    n_segments = (len(control_points) - 1) // 3
    t_values = np.linspace(0, n_segments, resolution+1)
    bezier_points = cubic_bezier_points_extended(control_points, t_values)
    distance = np.sqrt(np.sum(np.power(bezier_points[:-1,:] - bezier_points[1:,:],2),axis=-1))
    return distance

def cubic_bezier_points_equdistant(control_points, count=20, resolution=1000):
    n_segments = (len(control_points) - 1) // 3
    x = np.linspace(0, n_segments, resolution)
    y = numeric_distance_integration(control_points, resolution=resolution)
    length = np.sum(y)
    t_values_equidistant = np.interp(np.linspace(0, 1, count),y.cumsum()/length,x,)
    return cubic_bezier_points_extended(control_points, t_values_equidistant)

In [ ]:
control_points = np.array([
    [1, 1, 2],
    [8, 8, 2],
    [8, 5, 8],
    [8, 2, 8],
    [7, 2, 2],
    [2, 5, 8],
    [5, 5, 5],
])
bezier_points = cubic_bezier_points_equdistant(control_points, 30)
plot_bezier(control_points, bezier_points)